# 번개장터 데이터 전처리

In [1]:
import pandas as pd

In [2]:
original_path = "C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\\raw\\bungaejangter.csv"
csv_path = "C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\\preprocessed\\bungaejangter.csv"

## csv 데이터 확인

In [3]:
df = pd.read_csv(original_path, encoding="utf-8", engine="python", on_bad_lines="warn")

In [4]:
df.head(1)

,keyword,is_completed,uploaded_date,link,location,id,title,detail,condition,price,images_url
0,유모차 스토케,False,2025-08-20 07:43:46.595629,https://m.bunjang.co.kr/products/330635984?q=%...,서울특별시 용산구 한강로동,356fcf58-2382-46c5-b404-397e30c5e2a5,스토케 트레일즈 유모차,출고가 150만원 이었습니다\n사진 그대로 구매시 30만원\n겨울용 패딩시트 제외시...,사용감 적음,200000.0,['https://media.bunjang.co.kr/product/33063598...


In [5]:
df.shape

(148, 11)

## 관계 없는 데이터 삭제

In [5]:
# 부가부 데이터 삭제
df = df[~df['title'].str.contains("부가부", na=False)].reset_index(drop=True)

In [10]:
not_stroller_pattern = "유모차 컵홀더|스케토 컵홀더|스토케 풋머프|스토케풋머프|스토케 풋워머|스토케 방한 풋머프|스토케 방한용품|스토케 방한커버|방한커버|방수커버|컵홀더|파라솔"
df = df[~df['title'].str.contains(not_stroller_pattern, na=False)]

In [8]:
# 방한 관련 키워드
winter_keywords = "풋머프|풋워머|워머|핸드머프|핸드워머|방한커버|방한용품|방수커버|방수패드|방한"
df = df[~df['title'].str.contains(winter_keywords, na=False)]

In [12]:
# 유모차가 없는 경우 확인
df_filtered = df[~df['title'].str.contains('유모차', na=False)]
df_filtered['title']

134    스토케 익스플로리 엑스 시그니처 블랙
140      스토케 익스플로리 v6 판매합니다
Name: title, dtype: object

In [13]:
df.to_csv(csv_path, index=0)

## 변수 정리

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   keyword        129 non-null    object 
 1   is_completed   129 non-null    bool   
 2   uploaded_date  129 non-null    object 
 3   link           129 non-null    object 
 4   location       92 non-null     object 
 5   id             129 non-null    object 
 6   title          129 non-null    object 
 7   detail         129 non-null    object 
 8   condition      129 non-null    object 
 9   price          129 non-null    float64
 10  images_url     129 non-null    object 
dtypes: bool(1), float64(1), object(9)
memory usage: 10.3+ KB


In [16]:
df["detail"] = df["detail"].str.replace("\n", "", regex=False)
df.to_csv(csv_path, index=0)

In [18]:
df["uploaded_date"] = pd.to_datetime(df["uploaded_date"])
df.to_csv(csv_path, index=0)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   keyword        129 non-null    object        
 1   is_completed   129 non-null    bool          
 2   uploaded_date  129 non-null    datetime64[ns]
 3   link           129 non-null    object        
 4   location       92 non-null     object        
 5   id             129 non-null    object        
 6   title          129 non-null    object        
 7   detail         129 non-null    object        
 8   condition      129 non-null    object        
 9   price          129 non-null    float64       
 10  images_url     129 non-null    object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(8)
memory usage: 10.3+ KB


## 모델명 추출

In [14]:
import pandas as pd
df = pd.read_csv(csv_path, encoding="utf-8", engine="python", on_bad_lines="warn")
origin_df = df.copy()

In [38]:
patterns = {
    "yoyo": "요요|yoyo",
    "explori": "익스플로리|xplory",
    "trailz": "트레일즈|trailz",
    "beat": "비트|beat",
    "crusi": "크루시|crusi",
    "scoot": "스쿠트|scoot",
}
has_yoyo = df['title'].str.contains(patterns['yoyo'], na=False, case=False) | df['detail'].str.contains(patterns['yoyo'], na=False, case=False)

has_explori = df['title'].str.contains(patterns['explori'], na=False, case=False) | df['detail'].str.contains(patterns['explori'], na=False, case=False)

has_trailz = df['title'].str.contains(patterns['trailz'], na=False, case=False) | df['detail'].str.contains(patterns['trailz'], na=False, case=False)

has_beat = df['title'].str.contains(patterns['beat'], na=False, case=False) | df['detail'].str.contains(patterns['beat'], na=False, case=False)

has_crusi = df['title'].str.contains(patterns['crusi'], na=False, case=False) | df['detail'].str.contains(patterns['crusi'], na=False, case=False)

has_scoot = df['title'].str.contains(patterns['scoot'], na=False, case=False) | df['detail'].str.contains(patterns['yoyo'], na=False, case=False)

In [39]:
df[has_yoyo | has_trailz | has_explori | has_beat | has_crusi | has_scoot].count()

keyword          34
is_completed     34
uploaded_date    34
link             34
location         26
id               34
title            34
detail           34
condition        34
price            34
images_url       34
model             0
dtype: int64

In [40]:
# 기본값 None으로 열 추가
df["model"] = None

# 각 행별로 매칭된 모델 모두 합치기
def find_models(row):
    models = []
    for model, pattern in patterns.items():
        if (pd.notna(row["title"]) and pd.Series([row["title"]]).str.contains(pattern, case=False).any()) \
           or (pd.notna(row["detail"]) and pd.Series([row["detail"]]).str.contains(pattern, case=False).any()):
            models.append(model)
    return ", ".join(models) if models else None

df["model"] = df.apply(find_models, axis=1)

In [41]:
df['model'].unique()

array(['trailz', 'explori', 'yoyo', None, 'beat'], dtype=object)

In [35]:
delux_pattern = '디럭스|delux'
has_delux = df['title'].str.contains(delux_pattern, na=False, case=False) | df['detail'].str.contains(delux_pattern, na=False, case=False)

basic_pattern = '절충형'
has_basic = df['title'].str.contains(basic_pattern, na=False, case=False) | df['detail'].str.contains(basic_pattern, na=False, case=False)

In [36]:
df[has_delux | has_basic].count()

keyword          79
is_completed     79
uploaded_date    79
link             79
location         56
id               79
title            79
detail           79
condition        79
price            79
images_url       79
model             0
dtype: int64

In [44]:
# 기본값 None
df["model_type"] = None  

# 조건에 따라 값 지정
df.loc[has_delux, "model_type"] = "디럭스"
df.loc[has_basic, "model_type"] = "절충형"

In [46]:
df.to_csv(csv_path, index=0)

# 결론

In [47]:
df = pd.read_csv(csv_path)
df.head(1)

,keyword,is_completed,uploaded_date,link,location,id,title,detail,condition,price,images_url,model,model_type
0,유모차 스토케,False,2025-08-20 07:43:46.595629,https://m.bunjang.co.kr/products/330635984?q=%...,서울특별시 용산구 한강로동,356fcf58-2382-46c5-b404-397e30c5e2a5,스토케 트레일즈 유모차,출고가 150만원 이었습니다사진 그대로 구매시 30만원겨울용 패딩시트 제외시 20만...,사용감 적음,200000.0,['https://media.bunjang.co.kr/product/33063598...,trailz,NaN


광고 및 이상 데이터 제거하니 총 129개의 데이터가 나옴

모델이 특정되는 데이터는 총 34개

모델 타입이 특정되는 데이터는 79개

모델 또는 모델 타입이 특정 되는 데이터는 103개

In [52]:
df[~df['model'].isna() | ~df['model_type'].isna()].count()

keyword          103
is_completed     103
uploaded_date    103
link             103
location          73
id               103
title            103
detail           103
condition        103
price            103
images_url       103
model             34
model_type        79
dtype: int64

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   keyword        129 non-null    object 
 1   is_completed   129 non-null    bool   
 2   uploaded_date  129 non-null    object 
 3   link           129 non-null    object 
 4   location       92 non-null     object 
 5   id             129 non-null    object 
 6   title          129 non-null    object 
 7   detail         129 non-null    object 
 8   condition      129 non-null    object 
 9   price          129 non-null    float64
 10  images_url     129 non-null    object 
 11  model          34 non-null     object 
 12  model_type     79 non-null     object 
dtypes: bool(1), float64(1), object(11)
memory usage: 12.3+ KB
